# Todo List

- [x] completer le fichier function_preprocess
- [x] importer le preprocess qui reste dans #3
- [x] dans function_preprocess, reprendre le traitement de failure_type
- [ ]
- [ ] 

_____________________________________________________________________________________________________________

In [1]:
FILE_PATH = '../data/predictive_maintenance_A.csv'

import pandas as pd

import sys
sys.path.insert(0, '../Src/')

from Function_preprocess import preprocess

df = pd.read_csv(FILE_PATH, index_col='UDI')

df_train, df_test, dict_target = preprocess(df, False, False)


In [2]:
df_train.shape


(6382, 12)

In [ ]:
df_train.head()

_______________________________________________________________________________________________________________________

### Import

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [ ]:
import warnings

warnings.filterwarnings(action='ignore')

In [ ]:
FILE_PATH = '../data/predictive_maintenance_A.csv'

### Read file data

In [ ]:
df = pd.read_csv(FILE_PATH, index_col='UDI')
df.head()

Delete rows with 'Random Failures' in type column with target == 0

In [ ]:
df_tmp = df[df['Target']==0]
df_tmp['Failure Type'].value_counts()

In [ ]:
idx = df_tmp[df_tmp['Failure Type'] == 'Random Failures'].index
df.drop(index=idx, axis=0, inplace=True)

Delete rows with 'No Failure' in type column with target == 1

In [ ]:
df_tmp = df[df['Target']==1]
df_tmp['Failure Type'].value_counts()

In [ ]:
idx = df_tmp[df_tmp['Failure Type']=='No Failure'].index
df.drop(index=idx, axis=0, inplace=True)

In [ ]:
df.shape

In [ ]:

#from tensorflow import keras
#from tensorflow.python.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.python.keras.utils import to_categorical
a =to_categorical(df['Failure Type'], num_classes=df['Failure Type'].nunique())
print(a)

# Data Formatting
## Conversion 'Failure Type' variable to discret value

In [ ]:
lst_failure = df['Failure Type'].unique()
dict_failure = {}
for i, failure in enumerate(lst_failure):
    dict_failure[failure] = i
df['Failure Type'].map(dict_failure)  

## List of categorical variables
Memorize features who are 'object' type, exclude target variables 

In [ ]:
cat_cols = df.select_dtypes(include='O').columns.to_list()
cat_cols.remove('Failure Type')
#cat_cols.append('Target')
cat_cols

## List of numerical variables
Memorize features who are 'numerical' type, exclude target variables 

In [ ]:
num_cols = df.select_dtypes(exclude='O').columns.to_list()
num_cols.remove('Target')
num_cols

## Preprocessing of variables

#### Pipeline creation

Define different preprocessing to dataset
1. OnehotEncoder process from list of variables in 'cat_cols'
2. StandardScaler process from list of variables in 'num_cols'
3. OrdinalEncoder process for 'Failure Type' variables

In [ ]:
ct = ColumnTransformer([
    ("onehot", OneHotEncoder(), cat_cols),
    ("scale", StandardScaler(), num_cols),
    ("fail_type", OrdinalEncoder(), ['Target'])
])


Apply preprocess transformer and create dataframe 'df_preprocess' 

In [ ]:
df.drop(columns=['Target'], inplace=True)
df.rename(columns={'Failure Type': 'Target'}, inplace=True)
df_preprocess = pd.DataFrame(ct.fit_transform(df), columns=ct.get_feature_names_out())


In [ ]:

df_preprocess.dropna(axis=0, inplace=True)

Create dictionnary with index and value of 'target' feature

In [ ]:
cat_=ct.named_transformers_['fail_type'].categories_
dict_target = {}
dict_target_inv = {}
for i, c in enumerate(cat_[0]):
    dict_target[c] = i
    dict_target_inv[i] = c


Split dataframe into dataset for training and dataset for validation

In [ ]:
X_, X_val, y_, y_val = train_test_split(
    df_preprocess.drop(columns=['fail_type__Target']), 
    df_preprocess['fail_type__Target'], 
    test_size=0.2,
    stratify=df_preprocess['fail_type__Target'],
    random_state=42)

In [ ]:
df_train = pd.concat([X_, y_], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)

Save dataframe

In [ ]:
df_train.to_csv('../data/predictive_maintenance_preprocess.csv')
df_val.to_csv('../data/predictive_maintenance_validation.csv')

Save dictionary of target

In [ ]:
with open('../data/dict_target.json', 'w') as f:
    json.dump(dict_target_inv, f)